## try building a gpytorch model

In [1]:
import gpytorch
import torch
from torch.optim import SGD, Adam
from torch.optim.lr_scheduler import MultiStepLR
from torch import nn

from src.models.base_cnn import BaseCNN
from src.data.data_loader import get_splits, create_dataloaders

import math

import tqdm

Get some data

In [2]:
X_train, y_train, X_test, y_test = get_splits()

Train X dimensions: (238624, 101, 4) Test X dimensions: (26513, 101, 4)


In [3]:
device = torch.device('cuda')
train_dataloader, test_dataloader, num_feats = create_dataloaders(X_train=X_train, y_train=y_train, X_test=X_test, y_test=y_test, device=device, test_batch_size=10)

define a model

In [4]:
class BaseCNN(nn.Module):
    def __init__(self, seq_len: int = 101, 
                       dropout_prob: float = 0.15,
                       MLP_out_dim: int = 50) -> None:
        super().__init__()

        # configs
        self.dropout_prob = dropout_prob
        self.seq_len = seq_len

        # layers
        self.conv1 = nn.Conv1d(in_channels=4, out_channels=self.seq_len, kernel_size=4)
        self.pool1 = nn.MaxPool1d(kernel_size=3)
        self.conv2 = nn.Conv1d(in_channels=self.seq_len, out_channels=self.seq_len//2, kernel_size=4)
        self.pool2 = nn.MaxPool1d(kernel_size=3)
        self.dense = nn.Linear(in_features=450, out_features=MLP_out_dim)
        self.output = nn.Linear(in_features=MLP_out_dim, out_features=1)
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = x.transpose(1, 2).double()
        x = self.conv1(x)
        x = nn.functional.relu(x)
        x = self.pool1(x)
        
        x = self.conv2(x)
        x = nn.functional.relu(x)
        x = self.pool2(x)
        x = nn.functional.dropout(x, p=self.dropout_prob)        

        x = x.reshape((x.size(0), -1))
        x = self.dense(x)
        #x = nn.functional.relu(x)
        #x = nn.functional.dropout(x, p=self.dropout_prob)        

        #x = self.output(x)


        return x

In [15]:
# class GaussianProcessLayer(gpytorch.models.ApproximateGP):
#     def __init__(self, num_dim, grid_bounds=(-10., 10.), grid_size=64):
#         variational_distribution = gpytorch.variational.CholeskyVariationalDistribution(
#             num_inducing_points=grid_size, batch_shape=torch.Size([num_dim])
#         )

#         # Our base variational strategy is a GridInterpolationVariationalStrategy,
#         # which places variational inducing points on a Grid
#         # We wrap it with a IndependentMultitaskVariationalStrategy so that our output is a vector-valued GP
#         variational_strategy = gpytorch.variational.IndependentMultitaskVariationalStrategy(
#             gpytorch.variational.GridInterpolationVariationalStrategy(
#                 self, grid_size=grid_size, grid_bounds=[grid_bounds],
#                 variational_distribution=variational_distribution,
#             ), num_tasks=num_dim,
#         )
#         super().__init__(variational_strategy)

#         self.covar_module = gpytorch.kernels.ScaleKernel(
#             gpytorch.kernels.RBFKernel(
#                 lengthscale_prior=gpytorch.priors.SmoothedBoxPrior(
#                     math.exp(-1), math.exp(1), sigma=0.1, transform=torch.exp
#                 )
#             )
#         )
#         self.mean_module = gpytorch.means.ConstantMean()
#         self.grid_bounds = grid_bounds

#     def forward(self, x):
#         mean = self.mean_module(x)
#         covar = self.covar_module(x)
#         return gpytorch.distributions.Normal(mean, covar)

In [5]:
from gpytorch.models import ApproximateGP
from gpytorch.variational import CholeskyVariationalDistribution
from gpytorch.variational import VariationalStrategy

class GPModel(ApproximateGP):
    def __init__(self, inducing_points):
        variational_distribution = CholeskyVariationalDistribution(inducing_points.size(0))
        variational_strategy = VariationalStrategy(self, inducing_points, variational_distribution, learn_inducing_locations=True)
        super(GPModel, self).__init__(variational_strategy)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel())
        
    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

In [6]:
class ApproximateDKLRegression(gpytorch.Module):
    def __init__(self, inducing_points, num_dim=50, grid_bounds=(-1., 1.)):
        super(ApproximateDKLRegression, self).__init__()
        self.feature_extractor = BaseCNN()
        self.gp_layer = GPModel(inducing_points)
        self.grid_bounds = grid_bounds
        self.num_dim = num_dim

        # This module will scale the NN features so that they're nice values
        self.scale_to_bounds = gpytorch.utils.grid.ScaleToBounds(self.grid_bounds[0], self.grid_bounds[1])

    def forward(self, x):
        features = self.feature_extractor(x)
        features = self.scale_to_bounds(features)
        # This next line makes it so that we learn a GP for each feature
        #features = features.transpose(-1, -2).unsqueeze(-1)
        res = self.gp_layer(features)
        return res

In [7]:
mm = BaseCNN().double().cuda()
inducing_points = mm(next(iter(train_dataloader))[0].cuda())

In [ ]:
mm.train()
for epoch in range(100):
    running_loss = 0.0
    for batch in train_loader:
        optimizer.zero_grad()    
        examples, labels = batch     
    
        predictions = model(examples).reshape(-1)

        loss = criterion(predictions, labels.double())

        loss.double().backward()
        optimizer.step()

        running_loss += loss.item()


In [8]:
likelihood = gpytorch.likelihoods.GaussianLikelihood().to(device)
model = ApproximateDKLRegression(inducing_points).double().to(device)

In [17]:
n_epochs = 10
lr = 0.001
optimizer = Adam([
    {'params': model.feature_extractor.parameters()},
    {'params': model.gp_layer.hyperparameters(), 'lr': lr * 0.01},
    {'params': model.gp_layer.variational_parameters()},
    {'params': likelihood.parameters()},
], lr=lr)
scheduler = MultiStepLR(optimizer, milestones=[0.5 * n_epochs, 0.75 * n_epochs], gamma=0.1)
mll = gpytorch.mlls.VariationalELBO(likelihood, model.gp_layer, num_data=len(train_dataloader.dataset))


def train(epoch):
    model.train()
    likelihood.train()

    minibatch_iter = tqdm.notebook.tqdm(train_dataloader, desc=f"(Epoch {epoch}) Minibatch")
    with gpytorch.settings.num_likelihood_samples(8):
        for data, target in minibatch_iter:
            if torch.cuda.is_available():
                data, target = data.cuda(), target.cuda()
            optimizer.zero_grad()
            output = model(data)
            loss = -mll(output, target)
            loss.backward()
            optimizer.step()
            minibatch_iter.set_postfix(loss=loss.item())
        
def test():
    model.eval()
    likelihood.eval()

    means = torch.tensor([0.])
    test_y = torch.tensor([0.])
    with torch.no_grad(), gpytorch.settings.num_likelihood_samples(16):
        for data, target in test_dataloader:

            preds = model(data)
            means = torch.cat([means, preds.mean.cpu()])
            test_y = torch.cat([test_y, target.cpu()])
    means = means[1:]
    test_y = test_y[1:]
    print('Test MSE: {}'.format(torch.mean((means - test_y.cpu()) ** 2)))
    

In [18]:
for epoch in range(1, n_epochs + 1):
    with gpytorch.settings.use_toeplitz(False):
        train(epoch)
        test()
    scheduler.step()

(Epoch 1) Minibatch:   0%|          | 0/1865 [00:00<?, ?it/s]

Test MSE: 0.143752619293364


(Epoch 2) Minibatch:   0%|          | 0/1865 [00:00<?, ?it/s]

Test MSE: 0.14375604097763667


(Epoch 3) Minibatch:   0%|          | 0/1865 [00:00<?, ?it/s]

KeyboardInterrupt: 